In [1]:
import random
import math
import numpy as np
import copy
import getopt
import psycopg2
import sys
import time
import matplotlib.pyplot as plt

In [2]:
def LapNoise():
    a = random.uniform(0,1)
    b = math.log(1/(1-a))
    c = random.uniform(0,1)
    if c>0.5:
        return b
    else:
        return -b

In [14]:
num_user=[10000, 100000, 1000000, 10000000, 100000000]
ratio = 0.1
num_repeats = 20

fc = 0.54
fm = 0.37
fl = 0.09


In [10]:
def sample(eps, n, t):
    sum1 = sum(sum([eps>=t]))
    for i in range(n):
        if eps[i]<t:
            sum1+= np.random.binomial(1, (math.exp(eps[i])-1)/(math.exp(t)-1))
            
    return sum1+1/t*LapNoise()

In [31]:
times = []
errors = []
relative_error = []
real_results = []
for i in range(5):
    n = int(num_user[i]*ratio)
    
#     #Gaussian eps
#     #eps of non-zero users
#     eps = np.random.normal(1,0.3,int(n))
#     eps[eps<1/(10*math.sqrt(num_user[i]))]=1/(math.sqrt(10*num_user[i]))
#     # t = sum(eps)/n
#     eps_sort = np.sort(eps)

#     #to enable EM, we also need eps for zero users
#     eps_zero = np.random.normal(1,0.3,num_user[i]-int(n))
#     eps_zero[eps_zero<1/(10*math.sqrt(num_user[i]))]=1/(10*math.sqrt(num_user[i]))
#     eps_zero_sort = np.sort(eps_zero)
    
#     t = (sum(eps)+sum(eps_zero))/num_user[i]
#     eps_min = min(eps_sort[0], eps_zero_sort[0])
#     eps_max = max(eps_sort[n-1], eps_zero_sort[9*n-1])
    
    
    #Uniform eps
    nc = int(n*fc)
    nm = int(n*fm)
    nl = int(n*fl)
    epsc = np.random.uniform(1/(10*math.sqrt(num_user[i])), 0.5, nc)
    epsm = np.random.uniform(0.5, 2, nm)
    epsl = np.ones(nl)*2
    eps = np.append(np.append(epsc, epsm), epsl)
    eps_sort = np.sort(eps)
    #to enable EM, we also need eps for zero users
    nc_z = int((num_user[i]-n)*fc)
    nm_z = int((num_user[i]-n)*fm)
    nl_z = int((num_user[i]-n)*fl)
    epsc_z = np.random.uniform(1/(10*math.sqrt(num_user[i])), 0.5, nc_z)
    epsm_z = np.random.uniform(0.5, 2, nm_z)
    epsl_z = np.ones(nl_z)*2
    eps_zero = np.append(np.append(epsc_z, epsm_z), epsl_z)
    eps_zero_sort = np.sort(eps_zero)

    t = (sum(eps)+sum(eps_zero))/num_user[i]
    eps_min = min(eps_sort[0], eps_zero_sort[0])
    eps_max = max(eps_sort[n-1], eps_zero_sort[9*n-1])
    
    
    start = time.time()
    error = 0
    for j in range(num_repeats):
        result = sample(eps, n, t)
        error += abs(result-n)
    
    
    end= time.time()
    times.append((end-start)/num_repeats)
    errors.append(error/num_repeats)
    relative_error.append((error/num_repeats)/n)
    real_results.append(n)
#     print("Average Time for i = "+str(i))
#     print((end-start)/num_repeats)
#     print("Average Error for i = "+str(i))
#     print(error/num_repeats)

    
print("times")
print(times)
print("errors")
print(errors)
print("relative_error")
print(relative_error)
    

times
[0.0005015134811401367, 0.0038080215454101562, 0.0413670539855957, 0.40220558643341064, 3.9583815336227417]
errors
[425.6721022271477, 4172.993948976541, 41998.21093653537, 419140.3081466345, 4194978.086274575]
relative_error
[0.42567210222714774, 0.4172993948976541, 0.4199821093653537, 0.4191403081466345, 0.41949780862745756]


In [23]:
def pdp_count(eps, n, eps_min, eps_max, beta):
    t = math.ceil(math.log(eps_max/eps_min, 2))
    result=0
    for i in range(t+1):
        k = math.pow(2,i)*eps_min
#         print(sum(sum([eps>=k])))
#         print(t/k*np.log(t/beta))
        sum_tilde = sum(sum([eps>k]))+ t/k*LapNoise()-t/k*np.log(t/beta)
        result = max(result,sum_tilde)
    return result

In [24]:
times = []
errors = []
relative_error = []
real_results = []
for i in range(5):
    n = int(num_user[i]*ratio)

#     #Gaussian eps
#     #eps of non-zero users
#     eps = np.random.normal(1,0.3,int(n))
#     eps[eps<1/(10*math.sqrt(num_user[i]))]=1/(math.sqrt(10*num_user[i]))
#     # t = sum(eps)/n
#     eps_sort = np.sort(eps)

#     #to enable EM, we also need eps for zero users
#     eps_zero = np.random.normal(1,0.3,num_user[i]-int(n))
#     eps_zero[eps_zero<1/(10*math.sqrt(num_user[i]))]=1/(10*math.sqrt(num_user[i]))
#     eps_zero_sort = np.sort(eps_zero)
    
#     t = (sum(eps)+sum(eps_zero))/num_user[i]
#     eps_min = min(eps_sort[0], eps_zero_sort[0])
#     eps_max = max(eps_sort[n-1], eps_zero_sort[9*n-1])
    
    
    #Uniform eps
    nc = int(n*fc)
    nm = int(n*fm)
    nl = int(n*fl)
    epsc = np.random.uniform(1/(10*math.sqrt(num_user[i])), 0.5, nc)
    epsm = np.random.uniform(0.5, 2, nm)
    epsl = np.ones(nl)*2
    eps = np.append(np.append(epsc, epsm), epsl)
    eps_sort = np.sort(eps)
    #to enable EM, we also need eps for zero users
    nc_z = int((num_user[i]-n)*fc)
    nm_z = int((num_user[i]-n)*fm)
    nl_z = int((num_user[i]-n)*fl)
    epsc_z = np.random.uniform(1/(10*math.sqrt(num_user[i])), 0.5, nc_z)
    epsm_z = np.random.uniform(0.5, 2, nm_z)
    epsl_z = np.ones(nl_z)*2
    eps_zero = np.append(np.append(epsc_z, epsm_z), epsl_z)
    eps_zero_sort = np.sort(eps_zero)

    t = (sum(eps)+sum(eps_zero))/num_user[i]
    eps_min = min(eps_sort[0], eps_zero_sort[0])
    eps_max = max(eps_sort[n-1], eps_zero_sort[9*n-1])
    
    start = time.time()
    error = 0
    for j in range(num_repeats):
        result = pdp_count(eps, n, eps_min, eps_max, beta=0.1)
        error += abs(result-n)
    
    
    end= time.time()
    times.append((end-start)/num_repeats)
    errors.append(error/num_repeats)
    relative_error.append((error/num_repeats)/n)
    real_results.append(n)
#     print("Average Time for i = "+str(i))
#     print((end-start)/num_repeats)
#     print("Average Error for i = "+str(i))
#     print(error/num_repeats)

    
print("times")
print(times)
print("errors")
print(errors)
print("relative_error")
print(relative_error)
    

times
[0.0004590749740600586, 0.004633069038391113, 0.05408632755279541, 0.5561749935150146, 6.127076029777527]
errors
[533.7213979341278, 1378.4938122915728, 5837.349868020581, 18933.253065501456, 70818.47171790153]
relative_error
[0.5337213979341279, 0.13784938122915727, 0.05837349868020581, 0.018933253065501456, 0.007081847171790153]


In [4]:
def count_with_sampling(eps, n, eps_min, eps_max, beta):
    t = math.ceil(math.log(eps_max/eps_min, 2))
    result=0
    for i in range(t+1):
        k = math.pow(2,i)*eps_min
        sum_tilde = sum(sum([eps>k]))+ t/k*LapNoise()-t/k*np.log(t/beta)
        for i in range(n):
            if eps[i]<k:
                sum_tilde+= np.random.binomial(1, (math.exp(eps[i])-1)/(math.exp(k)-1))
                
        result = max(result,sum_tilde)
    return result

In [5]:
times = []
errors = []
relative_error = []
real_results = []
for i in range(5):
    n = int(num_user[i]*ratio)

#     #Gaussian eps
#     #eps of non-zero users
#     eps = np.random.normal(1,0.3,int(n))
#     eps[eps<1/(10*math.sqrt(num_user[i]))]=1/(math.sqrt(10*num_user[i]))
#     # t = sum(eps)/n
#     eps_sort = np.sort(eps)

#     #to enable EM, we also need eps for zero users
#     eps_zero = np.random.normal(1,0.3,num_user[i]-int(n))
#     eps_zero[eps_zero<1/(10*math.sqrt(num_user[i]))]=1/(10*math.sqrt(num_user[i]))
#     eps_zero_sort = np.sort(eps_zero)
    
#     t = (sum(eps)+sum(eps_zero))/num_user[i]
#     eps_min = min(eps_sort[0], eps_zero_sort[0])
#     eps_max = max(eps_sort[n-1], eps_zero_sort[9*n-1])
    
    
    #Uniform eps
    nc = int(n*fc)
    nm = int(n*fm)
    nl = int(n*fl)
    epsc = np.random.uniform(1/(10*math.sqrt(num_user[i])), 0.5, nc)
    epsm = np.random.uniform(0.5, 2, nm)
    epsl = np.ones(nl)*2
    eps = np.append(np.append(epsc, epsm), epsl)
    eps_sort = np.sort(eps)
    #to enable EM, we also need eps for zero users
    nc_z = int((num_user[i]-n)*fc)
    nm_z = int((num_user[i]-n)*fm)
    nl_z = int((num_user[i]-n)*fl)
    epsc_z = np.random.uniform(1/(10*math.sqrt(num_user[i])), 0.5, nc_z)
    epsm_z = np.random.uniform(0.5, 2, nm_z)
    epsl_z = np.ones(nl_z)*2
    eps_zero = np.append(np.append(epsc_z, epsm_z), epsl_z)
    eps_zero_sort = np.sort(eps_zero)

    t = (sum(eps)+sum(eps_zero))/num_user[i]
    eps_min = min(eps_sort[0], eps_zero_sort[0])
    eps_max = max(eps_sort[n-1], eps_zero_sort[9*n-1])
    
    start = time.time()
    error = 0
    for j in range(num_repeats):
        result = count_with_sampling(eps, n, eps_min, eps_max, beta=0.1)
        error += abs(result-n)
    
    
    end= time.time()
    times.append((end-start)/num_repeats)
    errors.append(error/num_repeats)
    relative_error.append((error/num_repeats)/n)
    real_results.append(n)
#     print("Average Time for i = "+str(i))
#     print((end-start)/num_repeats)
#     print("Average Error for i = "+str(i))
#     print(error/num_repeats)

    
print("times")
print(times)
print("errors")
print(errors)
print("relative_error")
print(relative_error)

times
[0.0073479413986206055, 0.033780574798583984, 0.294761061668396, 2.819723963737488, 30.799178957939148]
errors
[302.31010370244167, 1156.721716664617, 4137.503209716153, 12433.137555676512, 39514.815581053495]
relative_error
[0.30231010370244166, 0.1156721716664617, 0.04137503209716153, 0.012433137555676512, 0.00395148155810535]


In [27]:
def pdp_em_count(u):
    #first define the utility scores
    score = np.zeros(u+1)
    temp1 = 0
    temp2 = 0
    for i in range(u+1):
        if i<n:
            temp1+= eps_sort[i]
            score[n-1-i] = -temp1
            
        elif i>n:
            temp2+= eps_zero_sort[i-int(n)-1]
            score[i] = -temp2
#     print(score[n])
#     print(score[0])
#     print(score[num_user])
    weights = np.zeros(u+1)
    total_weight = 0
    for i in range(u+1):
        total_weight +=math.exp(1/2*score[i])
        weights[i] = math.exp(1/2*score[i])
    weights = weights/total_weight
#     print(weights[n])
    
    return np.random.choice(list(range(u+1)), p=weights)

In [37]:
times = []
errors = []
relative_error = []
real_results = []
for i in range(5):
    n = int(num_user[i]*ratio)

#     #Gaussian eps
#     #eps of non-zero users
#     eps = np.random.normal(1,0.3,int(n))
#     eps[eps<1/(10*math.sqrt(num_user[i]))]=1/(math.sqrt(10*num_user[i]))
#     # t = sum(eps)/n
#     eps_sort = np.sort(eps)

#     #to enable EM, we also need eps for zero users
#     eps_zero = np.random.normal(1,0.3,num_user[i]-int(n))
#     eps_zero[eps_zero<1/(10*math.sqrt(num_user[i]))]=1/(10*math.sqrt(num_user[i]))
#     eps_zero_sort = np.sort(eps_zero)
    
#     t = (sum(eps)+sum(eps_zero))/num_user[i]
#     eps_min = min(eps_sort[0], eps_zero_sort[0])
#     eps_max = max(eps_sort[n-1], eps_zero_sort[9*n-1])
    
    
    #Uniform eps
    nc = int(n*fc)
    nm = int(n*fm)
    nl = int(n*fl)
    epsc = np.random.uniform(1/(10*math.sqrt(num_user[i])), 0.5, nc)
    epsm = np.random.uniform(0.5, 2, nm)
    epsl = np.ones(nl)*2
    eps = np.append(np.append(epsc, epsm), epsl)
    eps_sort = np.sort(eps)
    #to enable EM, we also need eps for zero users
    nc_z = int((num_user[i]-n)*fc)
    nm_z = int((num_user[i]-n)*fm)
    nl_z = int((num_user[i]-n)*fl)
    epsc_z = np.random.uniform(1/(10*math.sqrt(num_user[i])), 0.5, nc_z)
    epsm_z = np.random.uniform(0.5, 2, nm_z)
    epsl_z = np.ones(nl_z)*2
    eps_zero = np.append(np.append(epsc_z, epsm_z), epsl_z)
    eps_zero_sort = np.sort(eps_zero)

    t = (sum(eps)+sum(eps_zero))/num_user[i]
    eps_min = min(eps_sort[0], eps_zero_sort[0])
    eps_max = max(eps_sort[n-1], eps_zero_sort[9*n-1])
    
    start = time.time()
    error = 0
    for j in range(num_repeats):
        result = pdp_em_count(num_user[i])
        error += abs(result-n)
    
    
    end= time.time()
    times.append((end-start)/num_repeats)
    errors.append(error/num_repeats)
    relative_error.append((error/num_repeats)/n)
    real_results.append(n)
#     print("Average Time for i = "+str(i))
#     print((end-start)/num_repeats)
#     print("Average Error for i = "+str(i))
#     print(error/num_repeats)

    
print("times")
print(times)
print("errors")
print(errors)
print("relative_error")
print(relative_error)

times
[0.0058525800704956055, 0.05031585693359375, 0.5198251008987427, 5.1977540254592896, 57.455408573150635]
errors
[134.5, 821.5, 1533.5, 3946.5, 11724.5]
relative_error
[0.1345, 0.08215, 0.015335, 0.0039465, 0.00117245]


In [15]:
##naive
times = []
errors = []
relative_error = []
real_results = []
for i in range(5):
    n = int(num_user[i]*ratio)

#     #Gaussian eps
#     #eps of non-zero users
#     eps = np.random.normal(1,0.3,int(n))
#     eps[eps<1/(10*math.sqrt(num_user[i]))]=1/(math.sqrt(10*num_user[i]))
#     # t = sum(eps)/n
#     eps_sort = np.sort(eps)

#     #to enable EM, we also need eps for zero users
#     eps_zero = np.random.normal(1,0.3,num_user[i]-int(n))
#     eps_zero[eps_zero<1/(10*math.sqrt(num_user[i]))]=1/(10*math.sqrt(num_user[i]))
#     eps_zero_sort = np.sort(eps_zero)
    
#     t = (sum(eps)+sum(eps_zero))/num_user[i]
#     eps_min = min(eps_sort[0], eps_zero_sort[0])
#     eps_max = max(eps_sort[n-1], eps_zero_sort[9*n-1])
    
    
    #Uniform eps
    nc = int(n*fc)
    nm = int(n*fm)
    nl = int(n*fl)
    epsc = np.random.uniform(1/(10*math.sqrt(num_user[i])), 0.5, nc)
    epsm = np.random.uniform(0.5, 2, nm)
    epsl = np.ones(nl)*2
    eps = np.append(np.append(epsc, epsm), epsl)
    eps_sort = np.sort(eps)
    #to enable EM, we also need eps for zero users
    nc_z = int((num_user[i]-n)*fc)
    nm_z = int((num_user[i]-n)*fm)
    nl_z = int((num_user[i]-n)*fl)
    epsc_z = np.random.uniform(1/(10*math.sqrt(num_user[i])), 0.5, nc_z)
    epsm_z = np.random.uniform(0.5, 2, nm_z)
    epsl_z = np.ones(nl_z)*2
    eps_zero = np.append(np.append(epsc_z, epsm_z), epsl_z)
    eps_zero_sort = np.sort(eps_zero)

    t = (sum(eps)+sum(eps_zero))/num_user[i]
    eps_min = min(eps_sort[0], eps_zero_sort[0])
    eps_max = max(eps_sort[n-1], eps_zero_sort[9*n-1])
    
    start = time.time()
    error = 0
    for j in range(num_repeats):
        result = n+1/eps_min*LapNoise()
        error += abs(result-n)
    
    
    end= time.time()
    times.append((end-start)/num_repeats)
    errors.append(error/num_repeats)
    relative_error.append((error/num_repeats)/n)
    real_results.append(n)
#     print("Average Time for i = "+str(i))
#     print((end-start)/num_repeats)
#     print("Average Error for i = "+str(i))
#     print(error/num_repeats)

    
print("times")
print(times)
print("errors")
print(errors)
print("relative_error")
print(relative_error)

times
[1.704692840576172e-06, 7.033348083496094e-07, 7.510185241699219e-07, 1.4662742614746095e-06, 1.2874603271484376e-06]
errors
[1341.5415017278451, 3032.4441740695956, 9524.769235018151, 41488.57730929395, 151657.52909588403]
relative_error
[1.3415415017278451, 0.30324441740695957, 0.09524769235018152, 0.04148857730929395, 0.015165752909588404]
